In [1]:
import requests
import time
import json
from bs4 import BeautifulSoup
import base64

ModuleNotFoundError: No module named 'pymongo'

In [5]:
hub_id = "8d2b1481-6bb6-44cd-81d5-c79a11c76fae"


In [3]:
def getHubInfo(id):
    return requests.get('https://api.faceit.com/hubs/v1/hub/8d2b1481-6bb6-44cd-81d5-c79a11c76fae').json()

In [5]:
hub = getHubInfo(hub_id)

In [78]:
def getHubMatches(page):
    params = {
        "id": "8d2b1481-6bb6-44cd-81d5-c79a11c76fae",
        "page": page,
        "size": 100,
        "type": "hub"
    }
    return get('https://api.faceit.com/match-history/v4/matches/competition', params)['payload']

In [92]:
def getPlayerMatches(id):
    params={
        "size": 100,
        "page": 0
    }
    return get(f"https://api.faceit.com/stats/v1/stats/time/users/{id}/games/csgo", params)

In [70]:
def getProxyList():
    soup = BeautifulSoup(requests.get('http://free-proxy.cz/en/proxylist/country/all/https/ping/level1').content, "lxml")
    rows = soup.find('table', id="proxy_list").find('tbody').findAll('tr')
    ips = []
    for row in rows:
        tds = row.findAll('td')
        try:
            coded = tds[0].text.strip()[30:][:-3]
            ip = base64.b64decode(coded).decode('utf-8')

            port = tds[1].text

            type_ = tds[2].text
    #         if type_ != "HTTPS": continue

            ips.append({ "https:": f"https://{ip}:{port}" })
        except:
            pass

    return ips

In [71]:
def get(url, params=None, proxies=None):
    tries = 1
    while True:
        res = requests.get(url, params=params, proxies=proxies)
        if res and res.status_code == 200:
            return res.json()
        elif res:
            print(f"Get error {res.status_code}")
        else:
            print("Get unknown error")
        tries+=1
        print("Sleeping for", tries * 10)
        time.sleep(tries + 10)

In [72]:
def parse(startPage=None, endPage=None):
    for page in range(450, 0, -1):
        print("Page", page)
        matches = getHubMatches(page)
        for match in matches:
            for i in range(1,2):
                print(i)
        
    

In [95]:
matches = getHubMatches(100)
for match in matches[:1]:
    for i in range(1,3):
        players = match['teams'][f"faction{i}"]['roster']
        for j, player in enumerate(players):
            matches = getPlayerMatches(player['id'])
            match['teams'][f"faction{i}"]['roster'][j]['matches'] = matches
    

{'matchId': '1-ad54511c-f4bc-49d5-b67c-6819d4263d2f', 'game': 'csgo', 'region': 'EU', 'winner': 'faction1', 'startedAt': '2020-07-23T03:40:48Z', 'finishedAt': '2020-07-23T04:20:38.608Z', 'state': 'finished', 'gameModeLabel': {'en': '5v5'}, 'gameModeType': '2Factions', 'teams': {'faction1': {'id': 'c2e350f3-4c42-4678-9f73-08af41d67435', 'leader': 'c2e350f3-4c42-4678-9f73-08af41d67435', 'name': 'team_--DoubleD', 'roster': [{'id': 'c2e350f3-4c42-4678-9f73-08af41d67435', 'nickname': '--DoubleD', 'gameId': '76561198061847210', 'gameName': 'train hater', 'avatar': 'https://assets.faceit-cdn.net/avatars/c2e350f3-4c42-4678-9f73-08af41d67435_1591632273303.jpg', 'gameSkillLevel': 10, 'acReq': False, 'matches': [{'_id': {'matchId': '5f64ac79f302c300069291d3', 'playerId': 'c2e350f3-4c42-4678-9f73-08af41d67435'}, 'created_at': 1600433273414, 'updated_at': 1600433364002, 'i9': '0', 'nickname': '--DoubleD', 'i10': '0', 'i13': '7', 'i15': '0', 'i6': '12', 'i14': '0', 'i7': '3', 'i16': '0', 'i8': '20',